### Check of all the data

In [2]:
import pandas as pd
import numpy as np 
from rapidfuzz import fuzz

In [3]:
wedstrijden = pd.read_excel('../../Data/Bronze/wedstrijden.xlsx')
corners = pd.read_excel('../../Data/Bronze/cornerballen2.xlsx')
wedstrijden_cleaned = pd.read_excel('../../Data/Silver/wedstrijden_cleaned.xlsx')
corners_cleaned = pd.read_excel('../../Data/Silver/cornerballen_cleaned.xlsx')


In [58]:
wedstrijden.tail()

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
217,218,2025-03-29 00:00:00,n1,8v4,NaN,3v5
218,219,2025-03-14 00:00:00,n1,5v4,NaN,1v5
219,220,2024-10-04 00:00:00,n1,9v6,NaN,2v3
220,221,2025-02-07 00:00:00,n1,7v4,NaN,11v12
221,222,2024-11-08 00:00:00,n1,2v6,NaN,4v2


In [59]:
corners.tail()

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
566,567,220,erwetegem,5.0,5.0,5.0,1,4,0,0,9v6,1,2,0,0,0,1
567,568,221,geraardsbergen a,5.0,5.0,5.0,1,3,0,0,4v1,1,2,0,0,1,0
568,569,222,erwetegem,5.0,5.0,5.0,0,1,1,0,0v0,1,2,0,0,0,1
569,570,222,deftinge,5.0,5.0,5.0,1,2,0,0,0v2,1,2,0,0,0,1
570,571,222,erwetegem,5.0,5.0,5.0,0,3,1,0,2v4,1,2,1,0,1,0


In [60]:
### check if there are teams with similar names

team_names = corners["ploegnaam"].unique()

# Find similar names
similar_names = []
threshold = 70  # similarity percentage threshold (can be lowered to catch more)

for i, name in enumerate(team_names):
    for j in range(i+1, len(team_names)):
        other_name = team_names[j]
        score = fuzz.ratio(name.lower(), other_name.lower())
        if score > threshold:
            similar_names.append((name, other_name, score))

# Display results
for name1, name2, score in similar_names:
    print(f"{name1} <-> {name2} : Similarity = {score}")


poba juniors <-> boca juniors : Similarity = 83.33333333333334
fc swakkes <-> fc swavey : Similarity = 73.6842105263158
het gouden hoofd a <-> gouden hoofd b : Similarity = 81.25
bbc united <-> ecp united : Similarity = 80.0
ecp united <-> eija united : Similarity = 76.19047619047619
the oliver boys <-> the oliver boys b : Similarity = 93.75
maldegem <-> maldegem b : Similarity = 88.88888888888889
drongen c <-> drongen : Similarity = 87.5


### correct similar names in the excel, so there are not 2 teams where there is actually just one

In [11]:
merged_df = pd.merge(wedstrijden, corners, on='wedstrijd_id', how='left')
merged_df.head()

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,...,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
0,1,2025-03-15 00:00:00,4,4v1,2v1,1v5,1,marathon,4.0,4.0,...,1,1,0,0v0,1,5,0,0,0,1
1,1,2025-03-15 00:00:00,4,4v1,2v1,1v5,2,marathon,4.0,4.0,...,2,0,0,2v0,1,5,0,0,0,0
2,1,2025-03-15 00:00:00,4,4v1,2v1,1v5,3,poba juniors,3.0,2.0,...,3,0,0,2v0,0,2,0,0,1,0
3,2,2024-09-03 00:00:00,4,11v2,2v1,8v3,4,mvc rudie,4.0,5.0,...,3,1,0,4v1,1,5,0,1,1,0
4,2,2024-09-03 00:00:00,4,11v2,2v1,8v3,5,uncle abes patty pounders,1.0,5.0,...,3,1,0,5v2,1,5,0,0,0,0


In [62]:
### check if scores are all feasible (eindresultaat lager dan een score voor een schepcorner)
feasible_check_df = merged_df[["wedstrijd_id", "uitslag", "is_thuisploeg", "score_voor", "goal"]]
feasible_check_df.head()

,wedstrijd_id,uitslag,is_thuisploeg,score_voor,goal
0,1,4v1,1,0v0,1
1,1,4v1,1,2v0,0
2,1,4v1,0,2v0,0
3,2,11v2,1,4v1,0
4,2,11v2,0,5v2,0


In [4]:
wedstrijd_ids_to_check = []

def parse_score(score_str):
    """Split score like '2v1' into (2,1) as integers."""
    parts = score_str.split("v")
    return int(parts[0]), int(parts[1])


In [5]:
def check_infeasible_scores(df):
    """Check is any score_voor not possible, given the uitslag."""
    wedstrijd_ids_to_check = []
    
    for i, row in df.iterrows():
        score_voor_home, score_voor_away = parse_score(row["score_voor"])
        uitslag_home, uitslag_away = parse_score(row["uitslag"])

        # Simulate goal being added
        if row["goal"] == 1:
            if row["is_thuisploeg"] == 1:
                score_voor_home += 1
            else:
                score_voor_away += 1

        # Check if the intermediate score exceeds the final result
        if score_voor_home > uitslag_home or score_voor_away > uitslag_away:
            wedstrijd_ids_to_check.append(row["wedstrijd_id"])

    # Remove duplicates (optional)
    return list(set(wedstrijd_ids_to_check))

In [65]:
wedstrijd_ids_to_check = check_infeasible_scores(feasible_check_df)

In [66]:
wedstrijd_ids_to_check.sort()

In [67]:
wedstrijd_ids_to_check

[28, 59, 78, 79, 83, 87, 95, 104, 118, 125, 128, 130, 178, 204, 212, 214, 220]

In [68]:
merged_df[merged_df["wedstrijd_id"].isin(wedstrijd_ids_to_check)][["datum","wedstrijd_id", "uitslag", "is_thuisploeg", "score_voor", "goal", "ploegnaam"]]

,datum,wedstrijd_id,uitslag,is_thuisploeg,score_voor,goal,ploegnaam
66,2024-08-29 00:00:00,28,6v4,0,3v10,1,sporting sakowa
148,2024-08-26 00:00:00,59,3v9,0,5v9,1,boca juniors
196,2024-10-03 00:00:00,78,4v11,0,2v3,1,atom solar
197,2024-10-03 00:00:00,78,4v11,1,2v6,1,de pallieterkes
198,2024-10-03 00:00:00,78,4v11,1,11v4,0,de pallieterkes
199,2024-09-20 00:00:00,79,4v5,0,5v3,1,united boys
206,2024-09-11 00:00:00,83,4v3,0,2v1,1,mvc bernies
207,2024-09-11 00:00:00,83,4v3,1,2v4,0,buzze bears
216,2024-09-10 00:00:00,87,4v2,0,1v1,1,tuacis
217,2024-09-10 00:00:00,87,4v2,1,1v3,0,los pepes


### redenen voor fouten
1) veel omwisseling score uit en thuis ploeg, zowel bij uitslag als bij score_voor
2) verkeerd wedstrijd_id bij een match
3) leesfouten uit geschrift
4) fout ingevulde enquetes

In [6]:
merged_cleaned_df = pd.merge(wedstrijden_cleaned, corners_cleaned, on='wedstrijd_id', how='inner')

In [7]:
wedstrijd_ids_cleaned_to_check = check_infeasible_scores(merged_cleaned_df)


In [8]:
wedstrijd_ids_cleaned_to_check

[]

In [9]:
### now lets check if the scores during the games make sense
def get_corrected_score(row):
    x, y = parse_score(str(row["score_voor"]))
    if row["goal"] == 1:
        if row["is_thuisploeg"] == 1:
            x += 1
        else:
            y += 1
    return x, y
    

In [10]:
def find_invalid_score_progression(df):
    """
    Check for invalid score progression within each match.
    Returns a DataFrame of problematic rows.
    """
    # Copy original dataframe and compute corrected scores
    df_copy = df.copy()
    df_copy[["score_home", "score_away"]] = df_copy.apply(get_corrected_score, axis=1, result_type="expand")

    invalid_rows = []

    # Check if score progression is increasing or constant
    for match_id, group in df_copy.groupby("wedstrijd_id"):
        previous_score = (-1, -1)
        for idx, row in group.iterrows():
            current_score = (row["score_home"], row["score_away"])
            if current_score < previous_score:
                invalid_rows.append(idx)
            previous_score = current_score

    # Filter and return problematic rows
    invalid_df = df_copy.loc[invalid_rows]
    print(f"Found {len(invalid_df)} rows where score progression is invalid.")
    return invalid_df[["datum", "wedstrijd_id", "uitslag", "is_thuisploeg", "score_voor", "goal", "ploegnaam"]]


In [74]:
invalid_progression_df = find_invalid_score_progression(merged_df)
display(invalid_progression_df)


Found 9 rows where score progression is invalid.


,datum,wedstrijd_id,uitslag,is_thuisploeg,score_voor,goal,ploegnaam
20,2024-09-02 00:00:00,8,2v15,0,1v6,0,mbapparte fc
99,2024-08-28 00:00:00,40,4v4,0,2v2,1,ecp united
285,2024-10-12 00:00:00,111,4v3,1,0v1,1,maptiko
321,2024-08-29 00:00:00,125,3v8,0,7v3,1,de pallieterkes
331,2024-08-27 00:00:00,130,4v2,0,1v4,0,de daltons
338,2024-08-26 00:00:00,134,6v8,0,0v1,1,elemax bvba
385,2024-09-24 00:00:00,150,3v2,1,1v2,0,thomas cook sport
388,2024-10-09 00:00:00,151,2v10,0,1v4,0,mvc doeffinge
535,2024-10-23 00:00:00,208,4v4,0,3v4,0,omerta


In [11]:
invalid_progression_cleaned_df = find_invalid_score_progression(merged_cleaned_df)
display(invalid_progression_cleaned_df)

Found 0 rows where score progression is invalid.


,datum,wedstrijd_id,uitslag,is_thuisploeg,score_voor,goal,ploegnaam


### redenen
1) score omgewisseld
2) fout ingevulde enquete
3) al correct door eerdere correctie

### data per kolom correct

In [17]:
# ervaring checken
print(corners["ervaring_schepper"].unique())
print(corners["ervaring_kopper"].unique())
print(corners["ervaring_duo"].unique())

[ 4.  3.  1. nan  5.  2.]
[ 4.  2.  5.  3. nan  1.]
[ 4.  3.  1.  2. nan  5.]


In [18]:
# is_thuisploeg checken
print(corners["is_thuisploeg"].unique())

[1 0]


In [19]:
# kwart checken
print(corners["kwart"].unique())

[1 2 3 4 0 5]


In [22]:
corners[corners["kwart"] == 5]

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
252,253,98,los galaticos,NaN,NaN,NaN,0,5,1,0,6v3,1,5,1,1,1,0


In [25]:
print(corners_cleaned["kwart"].unique())

[1 2 3 4]


In [26]:
# check bank
print(corners["is_schepper_bank"].unique())
print(corners["is_kopper_bank"].unique())

[1 0]
[0 1 5]


In [27]:
corners[corners["is_kopper_bank"] == 5]

,schepcorner_id,wedstrijd_id,ploegnaam,ervaring_schepper,ervaring_kopper,ervaring_duo,is_thuisploeg,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
239,240,93,de sparta hazards,NaN,NaN,NaN,0,4,1,5,3v4,1,5,0,0,0,0


In [30]:
print(corners_cleaned["is_kopper_bank"].unique())

[0 1]


In [31]:
# check posities
print(corners["schepper_pos_rechts"].unique())
print(corners["positie"].unique())


[1 0]
[5 2 6 3 4 1]


In [33]:
# check verplaatsen
print(corners["kop_verplaatsen_achter"].unique())
print(corners["kop_veel_verplaatsen"].unique())
print(corners["kop_hor_verplaatsen"].unique())

[0 1]
[0 1]
[0 1]


In [34]:
# check goal
print(corners["goal"].unique())

[1 0]


In [4]:
# check niveau
print(wedstrijden["niveau"].unique())

[4 3 1 2 nan 'n' 'beker' 'n4' 'p2' 'n1']


In [6]:
wedstrijden[wedstrijden["niveau"] == "n"]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
99,100,20/09/024,n,6v5,2v3,2v11
136,137,2024-12-06 00:00:00,n,8v0,1v2,3v9
137,138,2024-12-13 00:00:00,n,7v3,0v3,9v10
141,142,2025-01-10 00:00:00,n,8v5,NaN,8v9
146,147,2024-10-04 00:00:00,n,5v6,2v0,6v5
154,155,2024-10-18 00:00:00,n,4v6,0v0,8v5


In [8]:
# n en n1 zijn hetzelfde
wedstrijden_cleaned.loc[wedstrijden_cleaned["niveau"] == "n", "niveau"] = "n1"
wedstrijden_cleaned[wedstrijden_cleaned["niveau"] == "n"]


,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag


In [9]:
wedstrijden[wedstrijden["niveau"] == "beker"]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
123,124,2024-09-06 00:00:00,beker,15v2,NaN,NaN


In [12]:
merged_df[merged_df["wedstrijd_id"] == 124]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,...,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
316,124,2024-09-06 00:00:00,beker,15v2,NaN,NaN,317,team django gent,5.0,5.0,...,2,0,0,7v0,1,2,0,0,0,1
317,124,2024-09-06 00:00:00,beker,15v2,NaN,NaN,318,team django gent,5.0,5.0,...,4,0,0,10v1,1,2,0,0,0,1
318,124,2024-09-06 00:00:00,beker,15v2,NaN,NaN,319,the oliver boys b,5.0,5.0,...,4,0,0,15v1,1,2,0,0,0,1


In [14]:
# verander beker door n4 aangezien dat beide ploegen in die competitie spelen
wedstrijden_cleaned.loc[wedstrijden_cleaned["wedstrijd_id"] == 124, "niveau"] = "n4"
wedstrijden_cleaned[wedstrijden_cleaned["niveau"] == "beker"]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag


In [15]:
wedstrijden_cleaned.loc[wedstrijden_cleaned["wedstrijd_id"] == 124]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
123,124,2024-09-06 00:00:00,n4,15v2,NaN,NaN


In [ ]:
# check na
wedstrijden[wedstrijden["niveau"].isna()]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag
87,88,2024-09-11 00:00:00,NaN,3v2,2v0,NaN


In [17]:
merged_df[merged_df["wedstrijd_id"] == 88]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag,schepcorner_id,ploegnaam,ervaring_schepper,ervaring_kopper,...,kwart,is_schepper_bank,is_kopper_bank,score_voor,schepper_pos_rechts,positie,kop_verplaatsen_achter,kop_veel_verplaatsen,kop_hor_verplaatsen,goal
218,88,2024-09-11 00:00:00,NaN,3v2,2v0,NaN,219,velodrome,NaN,NaN,...,3,0,0,2v1,1,5,0,0,0,1
219,88,2024-09-11 00:00:00,NaN,3v2,2v0,NaN,220,alflex,NaN,NaN,...,4,0,0,3v2,1,5,1,0,0,0
220,88,2024-09-11 00:00:00,NaN,3v2,2v0,NaN,221,velodrome,NaN,NaN,...,4,0,0,3v2,1,5,0,1,1,0


In [18]:
# spelen kern niveau 2
wedstrijden_cleaned.loc[wedstrijden_cleaned["wedstrijd_id"] == 88, "niveau"] = 2
wedstrijden_cleaned[wedstrijden_cleaned["niveau"].isna()]

,wedstrijd_id,datum,niveau,uitslag,corners_uitslag,klassement_uitslag


In [19]:
print(wedstrijden_cleaned["niveau"].unique())

[4 3 1 2 'n1' 'n4' 'p2']


In [21]:
# checkt of kolommen bestaan uit volgende format getalvgetal of na zijn
def check_format(value):
    if pd.isna(value):
        return True  
    if isinstance(value, str):
        parts = value.split("v")
        if len(parts) == 2 and all(part.isdigit() for part in parts):
            return True
    return False

In [ ]:
# alles is ok
print(wedstrijden["uitslag"].apply(check_format).all())
print(wedstrijden["corners_uitslag"].apply(check_format).all())
print(wedstrijden["klassement_uitslag"].apply(check_format).all())
print(corners["score_voor"].apply(check_format).all())

True
True
True
True


In [ ]:
### wegschrijven
#wedstrijden_cleaned.to_excel('../../Data/Silver/wedstrijden_cleaned.xlsx', index=False)
#corners_cleaned.to_excel('../../Data/Silver/cornerballen_cleaned.xlsx', index=False)